## final model :
(Test-4, from Accident_Classifier_Model_Selection.ipynb)

Model: resnet18

Learning Rate: 0.001

Batch Size: 32

Activation Function: LeakyReLU

Epochs: 10

In [1]:
# Final accident classifier script locked to best configuration (Test 4)

import os
import sys
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
from torchvision.models import resnet18

In [ ]:
# Paths
DATA_DIR = "/content/drive/MyDrive/unified_data"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Best configuration (from Test 4)
ARCH = "resnet18"
LR = 0.001
BATCH_SIZE = 32
ACTIVATION_FN = nn.LeakyReLU
EPOCHS = 10

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size], generator=generator)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Model definition
def get_model():
    model = resnet18(weights="DEFAULT")
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_features, 256),
        ACTIVATION_FN(),
        nn.Dropout(0.3),
        nn.Linear(256, 2)
    )
    return model.to(DEVICE)

model = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct, train_total = 0.0, 0, 0
    loop = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{EPOCHS}")
    for i, (images, labels) in loop:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (preds == labels).sum().item()

        avg_loss = train_loss / train_total if train_total > 0 else 0.0
        avg_acc = train_correct / train_total if train_total > 0 else 0.0
        loop.set_postfix(train_loss=avg_loss, train_acc=avg_acc)
        sys.stdout.flush()

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

            val_loss += loss.item() * images.size(0)
            val_total += labels.size(0)
            val_correct += (preds == labels).sum().item()

    val_acc = val_correct / val_total
    avg_val_loss = val_loss / val_total
    print(f" Epoch {epoch+1} Summary: Train Acc: {avg_acc:.4f}, Train Loss: {avg_loss:.4f}, Val Acc: {val_acc:.4f}, Val Loss: {avg_val_loss:.4f}")

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "accident_classifier_resnet18.pth")
print(" Model saved as accident_classifier_resnet18.pth")